In [6]:
if (!require(survRM2)) install.packages('survRM2')
library('survRM2') 
if (!require(mvtnorm)) install.packages('mvtnorm')
library('mvtnorm')
if (!require(survival)) install.packages('survival')
library('survival')
if (!require(nph)) install.packages('nph')
library('nph')
if (!require(foreach)) install.packages("foreach")
library('foreach') 
if (!require(doParallel)) install.packages("doParallel")
library('doParallel')
if (!require(ggplot2)) install.packages("ggplot2")
library('ggplot2')
if (!require(cowplot)) install.packages("cowplot")
library('cowplot')
library('simtrial')

n_cores <- detectCores()
cluster <- makeCluster(14) # How many cores we use
registerDoParallel(cluster)
# multi thread
invisible(clusterEvalQ(cluster, #import packages to parallel 
  {
  library('survRM2')
  library('mvtnorm')
  library('survival')
  library('nph')
  library("simtrial")
  library("foreach")
  }))

source('/home/r27user6/RMST_Code/Function.R')
clusterExport(cluster, "expo_gen_2stages")

# Use the RMST difference test in 2 stages design
#### Rejection region of Interim:
#### $\textcolor{lightgreen}{E(\tau_1) - C(\tau_1) > m_1}$
#### Rejection region of overall:
#### $ \textcolor{lightgreen}{E(\tau_1) - C(\tau_1) > m_1\ \&\  E(\tau_2)-C(\tau_2)>m_2 }$

#### Similar to Shan(2021) (https://doi.org/10.1016/j.conctc.2021.100732)
#### But this paper is single arm. We are double
#### We use the non-HR setting in Eaton(2020)
####  <span style="color:yellow">Early differenct: A hazard ratio of 0.67 until 15 months and then a hazard ratio of 1.2 
#### acc_time = 24, cen_time = 36, interim = 18, final $\tau = 48$

In [12]:
median_con <- 10 # month
lambda_H0 <- log(2)/median_con
HR1 <- 0.67
HR2 <- 1.2
sim_size <- 10000 
acc_time <- 24
cen_time <- 36
n <- 100  # Fix total sample size
interim <- 18
change_time <- 15
set.seed(2024)

data_C <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0,
                         dist = 'exp', cen_time = cen_time, arm = 0, interim = interim)
data_E_H0 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, 
                         dist = 'exp', cen_time = cen_time, arm = 1, interim = interim)                           
data_E_H1 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, 
                         dist = 'pcw_exp', cen_time = cen_time,HR1 = HR1, HR2 = HR2, 
                         change_time = change_time, arm = 1, interim = interim)

#calculate the RMST of each group in H0,H1
data_C_int <- data_C[ , c(2,3,1)]  
data_E_H0_int <- data_E_H0[ , c(2,3,1)]
data_E_H1_int <- data_E_H1[ , c(2,3,1)]
rmst_h0_int <- RMST_sim_cal(n = n,data_E = data_E_H0_int, data_C = data_C_int,tau = interim,sim_size = sim_size)
rmst_h1_int <- RMST_sim_cal(n = n,data_E = data_E_H1_int, data_C = data_C_int,tau = interim,sim_size = sim_size)

data_C_fin <- data_C[ , c(4,5,1)]
data_E_H0_fin <- data_E_H0[ , c(4,5,1)]
data_E_H1_fin <- data_E_H1[ , c(4,5,1)]
rmst_h0_fin <- RMST_sim_cal(n = n,data_E = data_E_H0_fin, data_C = data_C_fin,tau = 48,sim_size = sim_size)
rmst_h1_fin <- RMST_sim_cal(n = n,data_E = data_E_H1_fin, data_C = data_C_fin,tau = 48,sim_size = sim_size)

rmst_data <- rbind(rmst_h0_int, rmst_h1_int, rmst_h0_fin, rmst_h1_fin)

#### Search for best m1 ,m2 to meet the best power

In [ ]:
find_m_t <- function(m_low, rmst_data, search_times, search_step,
                     tar_a1, tar_pow1_low, tar_pow1_up, tar_a2, sim_size) {
  rmst_h0_int <- rmst_data[c(1,2) , ]
  rmst_h1_int <- rmst_data[c(3,4) , ]
  rmst_h0_all <- rmst_data[c(1,2,5,6) , ]
  rmst_h1_all <- rmst_data[c(3,4,7,8) , ]
  result_m1 <- c()
  result_m1 <- foreach(i = 1:search_times, .combine = 'cbind') %dopar% { 
        m1 = m_low + i * search_step
        result_t1 <- c()
        proc_h0 <- sum((rmst_h0_int[2, ] - rmst_h0_int[1, ] > m1))
        proc_h1 <- sum((rmst_h1_int[2, ] - rmst_h1_int[1, ] > m1))
        if (proc_h0/sim_size > 0 && proc_h0/sim_size < tar_a1 &&
            proc_h1/sim_size >= tar_pow1_low ) {
            mark_c <- c(m1,proc_h0/sim_size,proc_h1/sim_size)           
            }
        mark_c
        }

  powerful_m1 <- result_m1[, which(result_m1[3,] == max(result_m1[3,]))]
  #Find the most powerful m1,t1
  if (is.null(dim(powerful_m1))) {
      # Return NULL when something goes wrong
      return(NULL)
    }
  bestm1 <- powerful_m1[, which(abs(powerful_m1[1,]) == min(abs(powerful_m1[1,] ))) ]
  # Among these most powerful, find the m1 with smallest absolute value 
  m1 <- bestm1[1]
  
  result_fin <- c()
  result_fin <- foreach(i = 1:search_times, .combine = 'cbind') %dopar% { 
        m2 = m_low + i * search_step
        opt_alpha <- 1
        opt_power <- 0
        opt_m2 <- 0
        opt_mt2 <- c()
        proc_h0 <- sum((rmst_h0_all[2, ] - rmst_h0_all[1, ] > m1) &
                    (rmst_h0_all[4, ] - rmst_h0_all[3, ] > m2) )
        proc_h1 <- sum((rmst_h1_all[2, ] - rmst_h1_all[1, ] > m1) &
                    (rmst_h1_all[4, ] - rmst_h1_all[3, ] > m2))
        if (proc_h0/sim_size > 0 & proc_h0/sim_size < tar_a2 &
            proc_h1/sim_size >= opt_power) # return the best 
            {
            opt_alpha <- proc_h0/sim_size
            opt_power <- proc_h1/sim_size
            opt_m2 <- m2
            opt_mt <- c(opt_m2,opt_alpha,opt_power)
            }
        opt_mt
        }

  if (is.null(dim(result_fin))) {
    # Return NULL when something goes wrong
    return(NULL)
  }
  else {
    powerful_fin <- result_fin[, which(result_fin[3,] == max(result_fin[3,]))]
    best_result <- powerful_fin[, which(abs(powerful_fin[1,]) == min(abs(powerful_fin[1,] ))) ]
     # most powerful result with smallest |m2|
    return(data.frame(m1 = m1,
                  PET0 = 1 - bestm1[2],
                  PET1 = 1 - bestm1[3],
                  m2 = best_result[1],
                  alpha = best_result[2],
                  Power = best_result[3]
                  ))
  }
  
}

## <span style="color:yellow">Optimal trial ( Min(EN) )
#### Stage I (Interim = 198) type I error can be set and $PET_0=1-\alpha_1 $
#### Under each $\alpha_1$ we can get empirical $power_1 = 1-PET_1$
#### We can search for the best $\alpha_1$ to minimize EN reach the overall alpha and power.

In [11]:
PET0 <- 0.2

s1result <- RMST_sim_test(n = 100, data_C = data_C[ , c(2, 3 ,1)] , data_E = data_E_H1[ , c(2, 3, 1)],
                     tau = interim , sim_size = sim_size, alpha = 1-PET0, sided = 'greater')
power_1 <- s1result$test_result$Rejection
PET1 <- 1 - power_1

s2result_H0 <- RMST_sim_test(n = 100, data_C = data_C[ , c(4, 5, 1)] , data_E = data_E_H0[ , c(4, 5, 1)],
                     tau = 48 , sim_size = sim_size, alpha = 1 - PET0, sided = 'greater')
s2result_H1 <- RMST_sim_test(n = 100, data_C = data_C[ , c(4, 5, 1)] , data_E = data_E_H0[ , c(4, 5, 1)],
                     tau = 48 , sim_size = sim_size, alpha = 1 - PET0, sided = 'greater')
 





[1] 0.0499

In [10]:
power_1

[1] 0.3764